# PIC-SURE API use-case: quick analysis on Hematopoietic Cell Transplant for Sickle Cell Disease (HCT for SCD) data

This is a tutorial notebook aimed to get the user quickly up and running with the python PIC-SURE API. It covers the main functionalities of the API.

## PIC-SURE python API 
### What is PIC-SURE? 

As part of the BioData Catalyst initiative, the Patient Information Commons Standard Unification of Research Elements (PIC-SURE) platform has been integrating clinical and genomic datasets funded by the National Heart Lung and Blood Institute (NHLBI). 

Original data exposed through the PIC-SURE API encompasses a large heterogeneity of data organization underneath. PIC-SURE hides this complexity and exposes the different study datasets in a single tabular format. By simplifying the process of data extraction, it allows investigators to focus on downstream analysis and to facilitate reproducible sciences.


### More about PIC-SURE
The API is available in two different programming languages, python and R, enabling investigators to query the databases the same way using either language.


PIC-SURE is a larger project from which the R/python PIC-SURE API is only a brick. Among other things, PIC-SURE also offers a graphical user interface that allows researchers to explore variables across multiple studies, filter patients that match criteria, and create cohorts from this interactive exploration.

The python API is actively developed by the Avillach Lab at Harvard Medical School.

PIC-SURE API GitHub repo:
* https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-python-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-python-client

 -------   

# Getting your own user-specific security token

**Before running this notebook, please be sure to review the "Get your security token" documentation, which exists in the NHLBI_BioData_Catalyst [README.md file](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token). It explains about how to get a security token, which is mandatory to access the databases.**

# Environment set-up

### Pre-requisite
- python 3.6 or later
- pip: python package manager, already available in most system with a python interpreter installed ([pip installation instructions](https://pip.pypa.io/en/stable/installing/))

### Install Packages

Install the following:
* packages listed in the `requirements.txt` file (listed below, along with verison numbers)
* PIC-SURE API components (from GitHub)
    * PIC-SURE Adapter
    * PIC-SURE Client
    
### Install packages
The first step to using the PIC-SURE API is to install the packages needed. The following code installs the PIC-SURE API components from GitHub, specifically:
* PIC-SURE Client
* PIC-SURE Adapter
* BioData Catalyst PIC-SURE Adapter

In [ ]:
!cat requirements.txt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git@new-search

import PicSureBdcAdapter

Import all the external dependencies and user-defined functions stored in the `python_lib` folder.

In [ ]:
import json
from pprint import pprint

import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

from python_lib.utils import get_multiIndex_variablesDict, get_dic_renaming_vars, joining_variablesDict_onCol

##### Set the display parameters for tables and plots

In [ ]:
# Pandas DataFrame display options
pd.set_option("max.rows", 100)

# Matplotlib parameters options
fig_size = plt.rcParams["figure.figsize"]
 
# Prints: [8.0, 6.0]
fig_size[0] = 14
fig_size[1] = 8
plt.rcParams["figure.figsize"] = fig_size

font = {'weight' : 'bold',
        'size'   : 14}

plt.rc('font', **font)

## Connecting to a PIC-SURE resource

The following is required to get access to the PIC-SURE API:
* a network URL
* a user-specific security token

The following code specifies the network URL as the BioData Catalyst Powered by PIC-SURE URL and references the user-specific token saved as `token.txt`.

If you have not already retrieved your user-specific token, please refer to the "Get your security token" section of the [README.md](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token) file.

In [ ]:
# Uncomment production URL when testing in production
# PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
token_file = "token.txt"

with open(token_file, "r") as f:
    my_token = f.read()
    
bdc = PicSureBdcAdapter.Adapter(PICSURE_network_URL, my_token)

## Getting help with the PIC-SURE python API

Each of the objects in the PIC-SURE API have a `help()` method that you can use to get more information about its functionalities.

In [ ]:
bdc.help()

For example, the above output lists and briefly defines the four methods that can be used with the `bdc` resource. 

## Export full HCT for SCD data dictionary to CSV

In [ ]:
full_dict = bdc.useDictionary().dictionary().find()
full_df = full_dict.dataframe()
scd_df = full_df[full_df.studyId.str.contains('phs002385')]
scd_df.head()

## Querying and retrieving data

If you are interested in learning more about the available query methods, see the `1_PICSURE_API_101.ipynb` notebook. 

Let's say we are interested in the age at which patients from the following cohorts received their transplants:
* males
* patients with avascular necrosis
* patients that received their transplant after the year 1999

First we will find variables pertaining to sex and avascular necrosis. We can do this by searching for "Sex" and "Avascular necrosis" in the `simplified_name` column of `variablesDict`.

In [ ]:
sex_var = variablesDict.loc[variablesDict["simplified_name"] == "Sex", "name"].values[0]
avascular_necrosis_varname = variablesDict.loc[variablesDict["simplified_name"] == "Avascular necrosis", "name"].values[0]

In [ ]:
# Peek at the result for avascular necrosis
variablesDict.loc[variablesDict["simplified_name"] == "Avascular necrosis", "name"] 

Next, we can find the variable pertaining to "Year of transplant".

In [ ]:
yr_transplant_varname = variablesDict.loc[variablesDict["simplified_name"] == "Year of transplant", "name"].values[0]
yr_transplant_varname

Now we can create a new query and apply our filters to retrieve the cohort of interest.

In [ ]:
# Patients with avascular necrosis
my_query.select().add(avascular_necrosis_varname)
my_query.filter().add(avascular_necrosis_varname, "Yes")

In [ ]:
# Males
my_query.select().add(sex_var)
my_query.filter().add(sex_var, "Male")

In [ ]:
# Patients receiving transplants after 1999
my_query.select().add(yr_transplant_varname)
my_query.filter().add(yr_transplant_varname, min=2000)

Using this cohort, we can add the variable of interest: "Patient age at transplant, years"

In [ ]:
age_transplant_var = variablesDict.loc[variablesDict["simplified_name"] == "Patient age at transplant, years", "name"].values[0]
my_query.select().add(age_transplant_var)

## Retrieving the data

Once our query object is finally built, we use the `query.run()` function to retrieve the data corresponding to our query

In [ ]:
query_df = my_query.getResultsDataFrame().set_index("Patient ID")

In [ ]:
query_df.head()

Once the data has been retrieved as a dataframe, you can use python functions to conduct analyses and create visualizations, such as this:

In [ ]:
query_df[age_transplant_var].plot.hist(legend=None, 
                                       title= "Age when transplant received in males with avascular necrosis from 2000 to present", 
                                       bins=15)